    - Ollama -> See : https://ollama.com/download
  - For linux users :
    
    For this tutorial, run these instructions
    - Install Ollama : `curl -fsSL https://ollama.com/install.sh | sh`
    - Pull required models : 
        - `ollama pull mayflowergmbh/occiglot-7b-fr-en-instruct` #french llm model
        - `ollama pull sammcj/sfr-embedding-mistral:Q4_K_M` # decent embedding for this use case
    
    -------------------------------------------------------------------------------
  
    Additional informations about Ollama
    - To remove Ollama : https://github.com/ollama/ollama/blob/main/docs/linux.md
    - To stop ollama server : `systemctl stop ollama`
    - To restart server : `systemctl start ollama`
    
    -------------------------------------------------------------------------------
    Due to issues from Ollama's latest versions in weights update, we might want to install older versions of Ollama.
    To install **v.0.1.31** on Linux:
      - `curl -fsSL https://ollama.com/install.sh | sed 's#https://ollama.com/download/ollama-linux-${ARCH}${VER_PARAM}#https://github.com/ollama/ollama/releases/download/v0.1.31/ollama-linux-amd64#' | sh`
            
- Next steps
    - Add source for response provided by the chatbot (e.g., source: from 'Le Challenger' via 'Malijet'. To know more, here are some useful links: links...)
    - Improve model response (accuracy and precision)


In [1]:
from pathlib import Path
import os
import uuid
import pandas as pd
from langchain_community.llms import Ollama
from langchain_core.output_parsers import StrOutputParser
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import OllamaEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.vectorstores.chroma import Chroma
# from langchain_community.vectorstores import DocArrayInMemorySearch
# from langchain_openai.embeddings import OpenAIEmbeddings

In [2]:
%env OPENAI_API_KEY=sk-xxx

In [3]:
ARTICLE_SOURCE_FILE_PATH = Path().resolve().parent /"data" / "malijet" / "source.csv"
CHROMA_DB_PERSIST_PATH = Path().resolve().parent / "data" / "chroma_db"
MODEL_NAME = "mayflowergmbh/occiglot-7b-fr-en-instruct"

In [4]:
# get nb cpu
os.cpu_count()

In [5]:
system_role = "Tu es un expert sur les actualités du Mali et tu parles uniquement français (spécialisé en langue française)."
llm = Ollama(model=MODEL_NAME, system=system_role, num_thread=os.cpu_count()-6)
llm

## Testing Simple LLM discussion

In [6]:
llm.invoke("Cite moi les noms des présidents du Mali.")

In [7]:
llm.invoke("Quelle est la plus grande crise que la Mali a connue ?")

In [8]:
llm.invoke("Who is the most popular scientist in the world?")

## 2. Build RAG with CSV file

In [9]:
loader = CSVLoader(
    file_path=ARTICLE_SOURCE_FILE_PATH,
    csv_args={
        "delimiter": "\t"
    }
)

# load documents
data = loader.load()
data[:3] # three first documents

In [10]:
# 53 articles extracted
len(data), pd.read_csv(ARTICLE_SOURCE_FILE_PATH, sep="\t").shape[0]

## Let's see if the splitter is necessary

### Without splitter

In [11]:
def get_length_info(list_of_documents, splitters=None):
    
    if splitters is None:
        splitters = [' ', '.']
    
    ## search the content length statistics (nb characters)
    print('-'*10, "For character length", '-'*10)
    display(pd.Series([len(document.page_content) for document in list_of_documents]).describe())
    
    ## search the words length statistics (nb words)
    print('-'*10, "Length of words (nb characters) in the corpus", '-'*10)
    _res = list()
    for document in list_of_documents:
        _res += pd.Series(document.page_content.split(splitters[0])).apply(len).tolist()
    display(pd.Series(_res).describe())
    
    ## search the sentence length statistics (nb words)
    print('-'*10, "How many words in each doc", '-'*10)
    display(pd.Series([len(doc.page_content.split(splitters[0])) for doc in list_of_documents]).describe())
    
    ## search the sentence length statistics (nb words)
    print('-'*10, "For how many sentences in in each doc", '-'*10)
    display(pd.Series([len(doc.page_content.split(splitters[1])) for doc in list_of_documents]).describe())
    
    ## search the sentence length statistics (nb words)
    print('-'*10, "For sentences length in each doc", '-'*10)
    display(pd.Series([len(sentence) for document in list_of_documents for sentence in document.page_content.split('.')]).describe())
    
    return _res

In [12]:
res = get_length_info(data)

## With splitter

In [13]:
pd.Series(res).max(), pd.Series(res).quantile(.99)

In [14]:
# quantile = int(pd.Series([len(sentence) for document in data for sentence in document.page_content.split('.')]).quantile(.95))
quantile = 387 # first version
maxi_character_per_chunk = pd.Series([len(sentence) for document in data for sentence in document.page_content.split('.')]).max().astype(int)
maxi_character_per_words = 20 # over 16 to make sure to get all-content overlap
quantile, maxi_character_per_chunk, maxi_character_per_words

How do I determine how many characters form a meaningful chunk (i.e., understandable and self-sufficient information)?
We will assume that each sentence could be a meaningful chunk. The idea is to have a count distribution of sentences length in our corpus to make a quick decision.
- We might choose 400 (similar to the quantile q95) as chunk size to be more flexible. In this case, we stay with 387 as a chunk size
- Also, in this case, we'll set 20 as the maximum length of character possible in a sentence

In [15]:
## test the document splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=quantile, # selected quantile
    chunk_overlap=maxi_character_per_words,
    separators=["\n\n", "\n", ". ", " ", ""], # specify that sentence split (by dot) is more important than space & other
    keep_separator=False # drop the separators from the document after split
)
documents = text_splitter.split_documents(documents=data)
len(documents)

In [16]:
documents[8:20]

In [17]:
res = get_length_info(documents)

 We move from 4616 words maxi per doc to 79 words per doc. That might be relevant for the final framework of RAG.

In [18]:
[doc.page_content for doc in documents[:5]]

In [19]:
[doc.page_content for doc in documents[37:40]]

## Embedding and Vector store

In [20]:
# embeddings_llm = OllamaEmbeddings(model=MODEL_NAME) #mistral or Occiglot
# embeddings_llm = OllamaEmbeddings(model="mayflowergmbh/occiglot-7b-fr-en-instruct")
# embeddings_llm = OllamaEmbeddings(model="snowflake-arctic-embed")
# embeddings_llm = OpenAIEmbeddings()

embeddings_llm = OllamaEmbeddings(model="sammcj/sfr-embedding-mistral:Q4_K_M")

# Set few params if needed
embeddings_llm.show_progress = True
embeddings_llm.num_thread = os.cpu_count() - 8 # 8 in my case
# embeddings_llm.top_k = 10
# embeddings_llm.top_p = .5

embeddings_llm

In [21]:
# Exemple
documents[3], len(documents[3].page_content)

In [22]:
# test to embed a query
embeddings_llm.embed_query(documents[3].page_content)[:7], len(embeddings_llm.embed_query(documents[3].page_content))

## Selection of Vector Store

Thanks to this brand-new article from Google, we can safely choose any open source Vector Store, make it available to Google NFS Filestore and access it easily through mounting filestore in Cloud Run (see section 3): https://cloud.google.com/blog/products/serverless/introducing-cloud-run-volume-mounts?hl=en

In [23]:
# The embedding is a very large dimension of 4096, so this will take a real long time
# Database creation
# db = Chroma.from_documents(
#     documents=documents[:2],
#     embedding=embeddings_llm,
#     persist_directory=CHROMA_DB_PERSIST_PATH.as_posix(),
# )
# db

In [24]:
db = Chroma(
    persist_directory=CHROMA_DB_PERSIST_PATH.as_posix(),
    embedding_function=embeddings_llm,
)

persisted_ids = db.get()["ids"]

new_documents_to_embed_df = pd.DataFrame({
    "single_id": [str(uuid.uuid5(uuid.NAMESPACE_DNS, doc.page_content)) for doc in documents],
    "document": documents
})

# to keep only different documents (i.e. chunks)
new_documents_to_embed_df.drop_duplicates(subset="single_id", inplace=True)

# Keep only documents not already embedded
new_documents_to_embed_df = new_documents_to_embed_df.query(f"single_id not in {persisted_ids}")

if new_documents_to_embed_df.empty:
    print("No documents to embed")
else:
    print("Embedding documents...")
    display(new_documents_to_embed_df.head())
    db.add_documents(
        documents=new_documents_to_embed_df.document.tolist(), 
        embedding=embeddings_llm, 
        ids=new_documents_to_embed_df.single_id.tolist(), 
        persist_directory=CHROMA_DB_PERSIST_PATH.as_posix(),
    )


In [25]:
db_check = Chroma(
    persist_directory=CHROMA_DB_PERSIST_PATH.as_posix(),
    embedding_function=embeddings_llm,
)
db_check.get()

In [42]:
# db_check.get(include=['embeddings'])

In [62]:
# db_check.delete_collection()

In [23]:
# In memory vector store test!

# Took more than 36min for 46 documents only; wow, I interrupted with the keyboard!
# For model name as occiglot or mistral or sfr-embedding first on retrieval
# db = DocArrayInMemorySearch.from_documents(documents, embedding=embeddings_llm)

In [26]:
# for snowflake
# db2 = DocArrayInMemorySearch.from_documents(documents, embedding=embeddings_llm) 

In [27]:
# for openai
# db4 = DocArrayInMemorySearch.from_documents(documents, embedding=embeddings_llm)

In [28]:
# load db from disk
# db3 = Chroma(persist_directory=CHROMA_DB_PERSIST_PATH.as_posix(), embedding_function=embeddings_llm)
# db3

## Retriever for RAG

In [29]:
retriever = db.as_retriever(search_kwargs={"k": 10}) # Occiglot or sfr-embed
# retriever = db2.as_retriever(search_kwargs={"k": 10}) # snowflake
# retriever = db4.as_retriever(search_kwargs={"k": 10}) #openAI
# retriever = db3.as_retriever(search_kwargs={"k": 5})
retriever

In [30]:
# Test retriever
query = "AES" 
retriever.invoke(query), len(retriever.invoke(query))

In [31]:
# Prompt template
template = """
Réponds à la question uniquement grâce au contexte suivant et uniquement en langue française.
Si tu n'as pas de réponse explicite dans le contexte, réponds "Je n'ai pas assez d'informations pour répondre correctement".

Contexte : {context}

Question : {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [32]:
def format_docs(docs):
    print([d.page_content for d in docs])
    return "\n\n".join([d.page_content for d in docs])

chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [33]:
questions = [
    # "Qui est Oumar Diarra ?",
    # "Quels sont les actions de l'EUTM ?",
    # "Cite-moi les recommandations retenues lors du dialogue inter malien",
    # "Quand finit la mission de l'union européenne ?",
    # "Actualités sur la DIRPA",
    # "Parle-moi de l'agriculture au Mali",
    # "Que font les FAMA actuellement ?"
    
    "Parle moi du nouveau vérificateur général",
    "Résume moi en quelques points les dernières actualités maintenant",
    "Où en est la relation Mali et Russie ?",
    "Qui est Bassaro Haïdara ?",
    "Qu'est ce que Assimi a fait récemment ?",
    "Le dialogue inter malien est il terminé ?",
    "Qui sont les membres de l'AES ?",
    "Comment a été la journée du 1er Mai au Mali ?",
    "Donne moi la date la plus récente des informations dont tu disposes",
    "Qu'en est il de la crise sécuritaire au Mali ?",
    "La Belgique a t elle récemment collaborée avec le Mali ?" # question bonus (must return I dont know)
]
len(questions)

In [34]:
# for open source models
for q in questions:
    print(q)
    print(chain.invoke(q))
    print('-'*50, "\n")

## Récapitulatif

- 8 bons retrieval sur 11 questions [V, V, V, X, V, V, X, V, X, V, V] : V → True response and X → Wrong response
- 4 bonnes réponses sur 11 questions [X, V, V, X, V, X, X, X, X, X, V] :  V → True response and X → Wrong response
- Bonne réponse à la question Bonus !
- Questions potentielles à rajouter :
    - Quand est ce que Abdoulaye Diop rencontrera ses homologues ?
    -  Quoi de prévu le 28 juin à Bamako ?
    - Quelle célébrité le président a rencontré ?
    - À quand la transition civile au Mali ?

# Some results from OpenAIEmbeddings, far better result

In [40]:
# Result from OpenAI Embeddings
for q in questions:
    print(q)
    print(chain.invoke(q))
    print('-'*50, "\n")